# Statistical Data Analysis

Python is an efficient tool for implementing numerous statistical data analysis operations.

NumPy is really useful in manipulating data and providing mathematical and statistic tools -- we will discover some more today.

## Vocabulary

In data science, we do not use the terms `rows` and `columns` to refer to a dataset. 
* The rows are called **samples**. For example, in our PokémonGo dataset, each sample was a Pokémon appearance.
* The columns are called **features**. Each feature describe a particular aspect of the sample. In our Pokémon Go dataset, it was the appaered Time, Hour, or Minute. It was a boolean of if it was closeToWater or not. It was the current weather in the game. In other datasets (with different samples), it could be the length of a sentence, the frequency of a word, the number of legs of an animal, a creation date, a GPS location, ...
* The **dimension** of a dataset is its number of features. If a dataset have 3 dimensions, we can call it a *3-dimensional* dataset.


# Principal component analysis (PCA)

The main purpose of PCA is **dimensionality reduction**.

If we have a `d`-dimensional dataset, we can try to:
* Reduce it to 2 or 3 dimensional dataset, so we can visualize it, and hopefully understand the data better
* Reduce it to a `k`-dimensional dataset, with `k` < `d`, so the computational cost of machine learning algorithms is reduced (i.e. for classification).

The `k`-dimensional *subspace* is a **projection** of our `d`-dimensional dataset.

To reduces the dimension, one needs to understand the relation between each features. This can done by computing a **scatter matrix** or a **covariance matrix**. We will use them to select which features to keep in our projection.

Here, we will implement PCA from scratch so we better understand how this algorithm works. We will go through these steps (they will be explained in details on the go):

* Get all the `d`-dimensions (the features we want to reduce)
1. Get the *mean vector*, the mean of each feature
1. Compute the covariance matrix or the scatter matrix of the dataset
1. Compute their eigenvectors $(e_{1}, e_{2},...,e_{d})$ and corresponding eigenvalues $(λ_{1},λ_{2},...,λ_{d})$ (in French, the *vecteurs propres* and *valeurs propres*)  
> The eigenvalues tell us the *length* (or *magnitude*) of the eigenvectors, and is a value >= 0.  
> If all the eigenvalues are similar in length, it means our dataset is well represented by those features.  
> If some eigenvalues are really high, and some are close to zero, it means the latters are less informative. We might consider dropping those features and keep only the higher values to construct the our `k`-dimensional subspace. 
1. Visualize the eigenvectors on a 3D Plan
1. Sort the eigenvectors by decreasing eigenvalues and choose `k` eigenvectors with the largest eigenvalues to form a `d`×`k` dimensional matrix $W$
1. Project our `d`-dimensional dataset in the new `k`-dimensioanl subspace, created from the eigenvectors with the highest eigenvalues.


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

import random  #  <-- a new library has appeared!
from pprint import pprint  #  <-- this is a built-in function to "pretty print", useful to visualize datastructures

## Create a dataset

First, we will create a small dataset of `nb_cats` cat samples, with a few features for each cat.

To generate our dataset, we will use the function `random.choice(L)` that randomly choose an option in the list `L`. So we will list the options in lists in `possible_values`.

The built-in Python function `range()` gives a `generator` of values (kind of equivalent to a `list`, but with *lazy* generation of elements). 
* If you give it one argument, `range(stop)` will give you each value from 0 to `stop-1`, iterating by 1. 
* If you give it two arguments, `range(start, stop)` will give you values from `start` to `stop-1`, iterating by 1. 
* If you give it three arguments, `range(start, stop, step)` will give you values from `start` to `stop-1`, iterating by `step`.

The function `arange()` from NumPy works the same, but authorize the use of type `float` instead of `int` as arguments.

Source of the Cat sizes: https://www.cat-toure.com/pages/size-chart

In [ ]:
possible_values = {
    "EyeColor": ["cyan", "lime", "black", "sienna", "gold"],
    "CoatColor": ["orange", "black"],
    "ChestLength (cm)": range(28, 52),
    "Length (cm)": range(26, 42),
    "Weight (kg)": np.arange(2, 9, 0.5)
}

nb_cats = 10

cats = []
for i in range(nb_cats):
    cat_sample = []
    for col, values in possible_values.items():
        cat_sample.append(random.choice(values))
    print(f"Sample {i}:", cat_sample)
    cats.append(cat_sample)

print("\nResulting cats dataset:")
pprint(cats)  #  <-- look how we used pretty printing here. 
# Try to use a basic `print` to see how ugly it is otherwise.

## Creating functions

Instead of modifying the value `nb_cats` each time we want to generate a dataset with less or more cats, we can create a function to call later on.


### Good practice: Typing 

Type your functions so Python can warn you when the types you give in argument are not those wanted by your function.

Basic data types in Python are:
* Integers: `int`
* Floating-Point numbers: `float`
* Strings: `str`
* Boolean: `bool`

More types can be found in the built-in Python modules `typing`.

Basic data structures:
* Lists: `list` or `typing.List`. With the latter, you can define the elements inside the list. For example, `List[str]` is a list of String elements.
* Dictionaries: `dict` or `typing.Dict`. With the latter, you can define the types of the Keys and Values. For example, `Dict[str, int]` is a dictionary with Strings keys containing integer values. 

You can combine data types definitions. For example, `Dict[str, List[str]]` is a dictionary that takes Strings as keys, and a list of Strings elements as values.

Let's rearrange our code above as a typed function.

In [ ]:
random.seed(42)  # This seed allow us to have consistency.

def create_cats_dataset(nb_cats: int) -> pd.DataFrame:  # Here, we return a pd.DataFrame.
    cats = []
    for i in range(nb_cats):
        cat_sample = []
        for values in possible_values.values():
            cat_sample.append(random.choice(values))
        cats.append(cat_sample)
    return pd.DataFrame(cats, columns=possible_values.keys())

print("\nCat dataset 1:")
tiny_dataset = create_cats_dataset(5)
pprint(tiny_dataset)

print("\nCat dataset 2:")
small_dataset = create_cats_dataset(25)
pprint(small_dataset)

# Step 1: Normalisation

Before applying PCA, we need to **normalize** our data.

The simpliest definition of normalisation is scaling values compared to the average. For each value $x$

$${\displaystyle z= x-{\bar {x}}} $$

where $\bar {x}$ is the mean of the sample.

This way, values that are equal to the average will be equal to 0. 

First, let's select normalizable features - they are the ones with numeric values. 

We could also transform String or Boolean values in numerical values, but we will see that later.


In [ ]:
d = 3 # We take the 3 last features, because they are numerical. This is our initial `d`-dimensional dataset.

normalizable_columns = list(possible_values.keys())[-d:]  # Look how we used the Slicing syntax here!
print(normalizable_columns)

## 3D Visualization using Matplotlib

We have 3 numerical features here - let's visualize them in 3D!

It's easy, you just have to:

* import mplot3d `from mpl_toolkits import mplot3d`
* specify to `fig.add_subplot` that you want a `projection='3d'`
* gives 3 arguments to your plot

In [ ]:
fig = plt.figure(figsize=(18, 12))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(small_dataset[normalizable_columns[0]],
           small_dataset[normalizable_columns[1]],
           small_dataset[normalizable_columns[2]], 
           c=small_dataset["CoatColor"])

ax.set(xlabel=normalizable_columns[0],
       ylabel=normalizable_columns[1],
       zlabel=normalizable_columns[2])

plt.show()

Note: c=small_dataset["CoatColor"] works because colors "gold" and "black" are [recognized Matplotlib colors](https://matplotlib.org/3.1.0/gallery/color/named_colors.html).

Each color point is the fur color of the cat represented in this 3D view.

In the case of our `small_dataset`, the means are:

In [ ]:
mean_vector = small_dataset[normalizable_columns].mean()
mean_vector

In [ ]:
small_dataset[normalizable_columns]

Let's use the `apply` function from Pandas (you should be familiar with it now!) to substract each value from its average.

To keep our initial data from our transformed data, we will copy our data in a new DataFrame, `small_dataset_transformed`.

### Hint
Instead of defining a function only for this purpose, we can use a **lambda function**.

It's also called a *anonymous function*: it's a function without a name, that we define using

> lambda *arguments* : *expression*


For exemple

`lambda x: x + 5`

is equivalent to

`def plus_five(x):
    return x + 5`

In [ ]:
small_dataset_transformed = small_dataset.copy()

# TODO: Use the `apply` function from Pandas to substract each value from its average.
for col in normalizable_columns:  
    small_dataset_transformed[col] = small_dataset[col]

small_dataset_transformed

# Step 2: Compute the matrices


## Method 1: The Covariance Matrix

### Using `cov` from NumPy

From the documentation of the `cov` method

> Signature:
np.cov(
    m,
    y=None,
    rowvar=True,
    bias=False,
    ddof=None,
    fweights=None,
    aweights=None,
)
> Parameters  
> ----------  
> m : array_like  
>    A 1-D or 2-D array containing multiple variables and observations.  
>    Each row of `m` represents a variable, and each column a single  
>    observation of all those variables. Also see `rowvar` below.  
  
  
So, let's transform our Pandas DataFrame into an array-like with each row being a variable.

In [ ]:
small_dataset_matrix = small_dataset_transformed[normalizable_columns].values

# Pandas and NumPy can be used together: using `.values` on a DataFrame gives us an NumPy array.
print(type(small_dataset_matrix))
print(small_dataset_matrix.shape)

small_dataset_matrix

### Transpose

But the shape is incorrect: we have a shape (n_samples, n_features) whereas `np.cov` waits for a (n_features, n_samples) shape.

if `Array` is your NumPy array, you can use the function `np.transpose(Array)` or `Array.T` to transpose it.

In [ ]:
print(np.transpose(small_dataset_matrix))
print(small_dataset_matrix.T)

In [ ]:
cov_matrix = np.cov(small_dataset_matrix.T)
cov_matrix

In [ ]:
# TODO: Print the second column of the `cov_matrix`.

print(cov_matrix)

In [ ]:
# TODO: Now, transpose the cov_matrix.

print(cov_matrix)

It's normal if it gives you the same result. 

Every covariance matrix is **symetric**, so if $M$ is a covariance matrix, $^{t}M = M$. 

## Method 2: Scatter Matrix

You can choose between the two methods. 

The scatter matrix is defined as an estimation of covariance matrix. Usually, people use the Scatter Matrix only when the Covariance matrix can't be computed (or is too costly to).

Our Scatter Matrix, like our Covariance Matrix, will be of shape $(d, d)$ if there is $d$ variables.

The scatter matrix is the sum of the dot product of each sample.

### Reminder: Dot Product

The **dot product** is the *produit scalaire* in French.

If the vectors $\vec x$ and $\vec y$ have coordinates 
$(x_{1}, x_{2}, x_{3})$ and 
$(y_{1}, y_{2}, y_{3})$, their dot product is:
 
$${\displaystyle {\vec {x}}\cdot {\vec {y}}=x_{1}y_{1}+x_{2}y_{2}+x_{3}y_{3}}.$$

In matrix writing:

$$ {\displaystyle {\vec {x}}\cdot {\vec {y}} = 
{}^{t}X~Y= 
{\begin{pmatrix}x_{1}&x_{2}&x_{3}\end{pmatrix}}{\begin{pmatrix}y_{1}\\y_{2}\\y_{3}\end{pmatrix}}=
x_{1}y_{1}+x_{2}y_{2}+x_{3}y_{3}} 
$$


In [ ]:
scatter_matrix = np.zeros((d,d)) # Create a NumPy array of shape (d, d) filled of 0.

for i in range(small_dataset_matrix.shape[0]):
    scatter_matrix += (small_dataset_matrix[i].reshape(d,1)).dot((small_dataset_matrix[i].reshape(d,1)).T)

scatter_matrix

# Step 3: Compute the eigenvectors and eigenvalues


Again, NumPy come for the rescue by providing us the methods `np.linalg.eig` to compute the eigenvalues and eigenvectors.

In [ ]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(scatter_matrix)
eig_val_sc, eig_vec_sc

In [ ]:
# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_matrix)
eig_val_cov, eig_vec_cov

## Checking

If the eigenvalues and vectors computation is correct, it should statisfy the equation

$$Σv=λv$$

where  
$Σ$ = Covariance matrix  
$v$ = Eigenvector  
$λ$ = Eigenvalue  

To test the equality, we will use the method `np.testing.assert_array_almost_equal(x, y)`

In [ ]:
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,3).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv), 
                                         eig_val_sc[i] * eigv,
                                         decimal=2, 
                                         verbose=True)

## Visualize the `d` eigenvectors

In [ ]:
for i in range(len(eig_val_sc)):
    eigvec_sc = eig_vec_sc[:,i]
    eigvec_cov = eig_vec_cov[:,i]
    
    print(f"\nEigenvector {i}")
    print(f"  Eigenvector from scatter matrix: {eigvec_sc}")
    print(f"  Eigenvector from covariance matrix: {eigvec_cov}")
    print(f"  Eigenvalue from scatter matrix: {eig_val_sc[i]}")
    print(f"  Eigenvalue from covariance matrix: {eig_val_cov[i]}")
    print(f"  Scaling factor: {round(eig_val_sc[i]/eig_val_cov[i], 2)}")

In [ ]:
fig = plt.figure(figsize=(18, 12))
ax = fig.add_subplot(111, projection='3d')

# Plot our points, like before
ax.scatter(small_dataset_transformed[normalizable_columns[0]],
           small_dataset_transformed[normalizable_columns[1]],
           small_dataset_transformed[normalizable_columns[2]], 
           c=small_dataset["CoatColor"])

origin = [0, 0, 0] # Data were normalized, so everything is centered around 0

# Use `ax.quiver` to plot each vector (array)
for v, color in zip(eig_vec_sc.T, ['r', 'g', 'b']):
    ax.quiver(*origin, v[0], v[1], v[2], color=color)

ax.set(xlabel=normalizable_columns[0],
       ylabel=normalizable_columns[1],
       zlabel=normalizable_columns[2])

plt.title('Eigenvectors')

plt.show()

# Step 4: Choose the `k` higher eigenvalues

We just printed the vectors for each of our 3 features.

In order to project these 3 features onto a smaller subspace, which will be `k` = 2, we should keep the `k` eigenvectors with the highest eigenvalues.

In [ ]:
# Create a tuple of corresponding (eigenvalue, eigenvector)
eig_pairs = [(val, vec) for val, vec in zip(eig_val_cov, eig_vec_cov)]
eig_pairs

In [ ]:
sorted(eig_pairs, reverse=True)

In [ ]:
k = 2
k_eig_vectors = sorted(eig_pairs, reverse=True)[:k]  # Slicing syntax again
k_eig_vectors

Then, we use `np.hstack` to stack horizontally each selected eigenvectors to our new matrix $W$ `matrix_w`.

This matrix will be used to make our projection from `d`-dimensional datasets to a `k`-dimensional one.

In [ ]:
matrix_w = np.hstack([k_eig_vectors[0][1].reshape(d, 1), 
                      k_eig_vectors[1][1].reshape(d, 1)])
matrix_w

# Final step: Project onto the new subspace


For that, we need to create a (`d`,`k`) shaped matrix $W$.

We then transform our samples onto the new subspace via the equation $y = ^{t}Wx$

* $x$ being the samples `small_dataset`, of shape (`d`, n_samples).
* $y$ being our transformed samples of shape (`k`, n_samples).

In [ ]:
print(small_dataset_matrix.T.shape)

In [ ]:
# TODO: Project `small_dataset_matrix` in `k_subspace` by using a dot product on matrix_w.

## Hint: You can ctrl+f "dot" in this document to see previous usage.

k_subspace = small_dataset_matrix.T

print(k_subspace.shape)  # Should be (2, 25)
k_subspace

Your resulting shape should be `(k, n_samples)`.


It's `(2, 25)` if you didn't modify the variable `k` and the number of cats generated.

## Visualize data in the new subspace

We're finished! 

We just used the two vectors as axis for our space, and `k_subspace`

In [ ]:
fig, ax = plt.subplots(1, figsize=(18, 12))

plt.scatter(k_subspace[0], k_subspace[1], c=small_dataset["CoatColor"])

plt.title('small_dataset on the new subspace')

ax.set(xlabel="New X axis",
       ylabel="New Y axis")
       
plt.show()

# Wonderful!

Thank you for following this tutorial and doing your Principal Component Analysis almost by yourself.

I hope you understood how it works -- it's important to understand what algorithms are doing when we use them.

Now that you are familiar with it, I have good news for you.

The PCA is implemented in the `scikit-learn` library!

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(small_dataset_matrix)

principal_components

In [ ]:
fig, ax = plt.subplots(1, figsize=(18, 12))

plt.scatter(principal_components[:, 0], principal_components[:, 1], c=small_dataset["CoatColor"])

plt.title('small_dataset on the new subspace using sklearn')

ax.set(xlabel="New X axis",
       ylabel="New Y axis")

plt.show()